# Fast convolution tutorial
This tutorial is about implementing fast convolutions using im2col trick.<br>


# Tensor class

In [1]:
class Tensor:
    """
    Refer The below link to know why mutable datasetrutures cant be in the initialization
    https://stackoverflow.com/questions/4841782/python-constructor-and-default-value
    """
    def __init__(self,value,parent=None,child=None,operation=None,grad=None,gradstatus =0):
        self.value = value
        self.grad = grad
        self.operation = operation
        self.gradstatus = 0
        if child is None:
            self.child = []
        else:
            self.child = child
        if parent is None:
            self.parent = []
        else:
            self.parent = parent
            
        
    def get_value(self):
        return self.value
    def get_consumer(self):
        return self.child
    def get_parent(self):
        return self.parent
    def get_operation(self):
        return self.operation
    def get_grad(self):
        return self.grad
    def get_gradstatus(self):
        return self.gradstatus
    
    def update_child(self,child):
        self.child.append(child)
    def update_value(self,value):
        self.value = value
    def update_grad(self,grad):
        self.grad += grad
   
    def set_grad(self,grad):
        self.grad = grad
    def set_gradstatus(self,stat):
        self.gradstatus = stat
        
    def size(self):
        return 1
    def __repr__(self):
        return 'tensor object:'+ str(id(self)) 

# im2col made faster using 2d input and 2d output
Before importing the below im2col, col2im.. files you have to compile them running the below code in command prompt.
>python setup.py build_ext --inplace

You can see each of the codes corresponding speed by using the below code
>cython -a im2col_cython.pyx

>cython -a col2im_cython.pyx

>cython -a im2col6.pyx

The above code will generate html files, the yellowness depicting the time consumption, more yellow more time consumed in operation.

In [28]:
import im2col6 #from cs231n
import im2col_cython
import col2im_cython

class Convolution2D:
    import numpy as np
    def __init__(self,stride=2,pad=1):
        self.stride = stride
        self.pad = pad
        
    def __call__(self,X,w):
        res = Convolution2D.__conv_v5_3d__(X,w,self.stride,self.pad)
        return res
    
    
            
    def __get_input_value__(input1,input2):
        return (input1.get_value(),input2.get_value())
    
    def __pad__(X_val,pad):
        x_shp = X_val.shape
        X_tmp    = np.zeros((x_shp[0],x_shp[1],x_shp[2]+2*pad,x_shp[3]+2*pad))
        X_tmp[:,:,pad:(x_shp[2]+pad),pad:(x_shp[3]+pad)] = X_val
        return X_tmp
        #return np.pad(X_val,((0,0),(0,0),(pad,pad),(pad,pad)),'constant',constant_values=0) #very slow
    
    def __get_shape__(X_val,w_val):
        return (X_val.shape,w_val.shape)
    
    def __assert_shape__(x_shp,w_shp,stride):
        try:
            assert(x_shp[1]==w_shp[1]), 'Wrong channel,filter'
            assert((x_shp[2]-w_shp[2])%stride == 0), 'Wrong stride'
            assert((x_shp[3]-w_shp[3])%stride == 0), 'Wrong stride'
            return (x_shp[0],w_shp[0],int((x_shp[2]-w_shp[2])/stride+1),int((x_shp[3]-w_shp[3])/stride+1))
        except:
            print('Error: Incompatible Tensor dimensions for convolution')
            return 0

    def __rect_shape__(X_val,w_val):
        if(len(X_val.shape)==2):
            X_val.reshape(1,1,X_val.shape)
        if(len(w_val.shape)==3):
            w_val.reshape(1,w_val.shape)
        return X_val,w_val
    
    """
    # Naive implementatiom
    def __conv_v1__(X,w,stride,pad):
        X_val,w_val  = Convolution2D.__get_input_value__(X,w)
        X_val        = Convolution2D.__pad__( X_val ,pad)
        x_shp,w_shp  = Convolution2D.__get_shape__(X_val,w_val)
        res_shape    = Convolution2D.__assert_shape__(x_shp,w_shp,stride)
        X_res        = np.zeros(shape = res_shape)
        if res_shape:

            for i in range(res_shape[0]):
                for j in range(res_shape[1]):

                    row_idx = np.arange(i,x_shp[1]-w_shp[1]+i+1,stride)
                    col_idx = np.arange(j,x_shp[2]-w_shp[2]+j+1,stride)
                    X_res[i,j] = np.sum(np.multiply(X_val[:,i*stride:(i*stride+w_shp[1]),j*stride:(j*stride+w_shp[2])],w_val))


            return X_res
    """
    """
    # Smarter than above but still Naive
    def __conv_v2__(X,w,stride,pad):
        X_val,w_val  = Convolution2D.__get_input_value__(X,w)
        X_val        = Convolution2D.__pad__( X_val ,pad)
        x_shp,w_shp  = Convolution2D.__get_shape__(X_val,w_val)
        res_shape    = Convolution2D. __assert_shape__(x_shp,w_shp,stride)
        X_res        = np.zeros(shape = res_shape)
        if res_shape:
            for i in range(w_shp[1]):
                for j in range(w_shp[2]):
                    for l in range(w_shp[0]):
                        row_idx = np.arange(i,x_shp[1]-w_shp[1]+i+1,stride)
                        col_idx = np.arange(j,x_shp[2]-w_shp[2]+j+1,stride)
                        X_res += X_val[l,row_idx[:, None], col_idx]*w_val[l,i,j] 
            return X_res
    """
    
    #"""
    # Used for asserting answers obtained through vectorised implementation
    def __conv_v2_3d__(X,w,stride,pad):
        X_val,w_val  = Convolution2D.__get_input_value__(X,w)
        X_val        = Convolution2D.__pad__( X_val ,pad)
        x_shp,w_shp  = Convolution2D.__get_shape__(X_val,w_val)
        res_shape    = Convolution2D. __assert_shape__(x_shp,w_shp,stride)
        X_res        = np.zeros(shape = res_shape)
        if res_shape:
            for k in range(w_shp[0]):
                for i in range(w_shp[2]):
                    for j in range(w_shp[3]):
                        for l in range(w_shp[1]):

                            row_idx = np.arange(i,x_shp[2]-w_shp[2]+i+1,stride)
                            col_idx = np.arange(j,x_shp[3]-w_shp[3]+j+1,stride)
                            X_res[:,k,:,:] += X_val[:,l,row_idx[:, None], col_idx]*w_val[k,l,i,j] #0.19
            return X_res
    
    def __back_conv_v2_3d__(X,w,stride,pad,grad):
        X_val,w_val  = Convolution2D.__get_input_value__(X,w)
        X_val        = Convolution2D.__pad__( X_val ,pad)
        x_shp,w_shp  = Convolution2D.__get_shape__(X_val,w_val)
        res_shape    = Convolution2D. __assert_shape__(x_shp,w_shp,stride)
        X_rev        = np.zeros(shape = x_shp)
        w_rev        = np.zeros(shape = w_shp)
        if res_shape:
            for k in range(w_shp[0]):
                for i in range(w_shp[2]):
                    for j in range(w_shp[3]):
                        for l in range(w_shp[1]):
                            row_idx = np.arange(i,x_shp[2]-w_shp[2]+i+1,stride)
                            col_idx = np.arange(j,x_shp[3]-w_shp[3]+j+1,stride)
                            w_rev[k,l,i,j] = np.sum(grad[:,k,:,:]*X_val[:,l,row_idx[:, None], col_idx])
                            X_rev[:,l,row_idx[:, None], col_idx] += grad[:,k,:,:]*w_val[k,l,i,j]
            return X_rev,w_rev
    #"""
   
    def __conv_v5__(X,w,stride,pad):
        X_val,w_val  = Convolution2D.__get_input_value__(X,w)
        X_val        = Convolution2D.__pad__( X_val ,pad)
        x_shp,w_shp  = Convolution2D.__get_shape__(X_val,w_val)
        res_shp    = Convolution2D. __assert_shape__(x_shp,w_shp,stride)
        X_res        = np.zeros(shape = (res_shp[0],res_shp[1],w_shp[1]*w_shp[2]*w_shp[0]))
        if res_shp:
            iter = -1
            for l in range(w_shp[0]):   
                for i in range(w_shp[1]):
                    for j in range(w_shp[2]):
                        iter+=1
                        #print(iter)
                        row_idx = np.arange(i,x_shp[1]-w_shp[1]+i+1,stride)
                        col_idx = np.arange(j,x_shp[2]-w_shp[2]+j+1,stride)
                        X_res[:,iter] = np.reshape(X_val[l,row_idx[:, None], col_idx],(res_shp[0]*res_shp[1]))
            return np.reshape(np.dot(X_res,np.reshape(w_val,(w_shp[0]*w_shp[1]*w_shp[2],1))),(res_shp[0],res_shp[1]))
    

    def __conv_v5_3d__(X,w,stride,pad):
        t0 = time.time()
        X_val,w_val  = Convolution2D.__get_input_value__(X,w)
        t1 = time.time()
        print('Preliminary : getvalue ',t1-t0)
        t0 = time.time()
        X_val        = Convolution2D.__pad__( X_val ,pad)
        t1 = time.time()
        print('Preliminary : pad ',t1-t0)
        t0 = time.time()
        x_shp,w_shp  = Convolution2D.__get_shape__(X_val,w_val)
        t1 = time.time()
        print('Preliminary : getshape ',t1-t0)
        t0 = time.time()
        res_shp    = Convolution2D. __assert_shape__(x_shp,w_shp,stride)
        t1 = time.time()
        print('Preliminary : assert shape ',t1-t0)
        
        
        
        if res_shp:
         
            
            """
            # CS231N im2col
            t0 = time.time()
            X_res = im2col6.im2col_cython(X_val,w_shp[2],w_shp[3], 0,stride)
            t1 = time.time()
            X_res = X_res.T
            print('Cython way im2col:',t1-t0)
            
            t0 = time.time()
            res=  np.dot(X_res,w_val.reshape(w_shp[0],w_shp[1]*w_shp[2]*w_shp[3]).T)
            t1 = time.time()
            print('Dot product:',t1-t0)
            
            t0 = time.time()
            res = res.reshape(res_shp[2]*res_shp[3],x_shp[0],w_shp[0])
            Res = res.swapaxes(0,1).swapaxes(1,2).reshape(res_shp)
            t1 = time.time()
            print('Final Swapping,reshaping:',t1-t0)
            """
            #"""
            # My code
            t0 = time.time()
            X_res = im2col_cython.im2col_2d(np.array(w_shp),np.array(x_shp),stride,X_val.reshape(x_shp[0]*x_shp[1]*x_shp[2],x_shp[3]))
            
            t1 = time.time()
            print('Cython way im2col:',t1-t0)
            
            t0 = time.time()
            res=  np.dot(X_res,w_val.reshape(w_shp[0],w_shp[1]*w_shp[2]*w_shp[3]).T)
            t1 = time.time()
            print('Dot product:',t1-t0)
            
            t0 = time.time()
            res = res.reshape(x_shp[0],res_shp[2]*res_shp[3],w_shp[0])
            Res = res.swapaxes(1,2).reshape(res_shp)
            t1 = time.time()
            print('Final Swapping,reshaping:',t1-t0)
            #"""
            
            return Res
        
    def __back_conv_v5_3d__(X,w,stride,pad,grad):
        
        X_val,w_val  = Convolution2D.__get_input_value__(X,w)
        X_val        = Convolution2D.__pad__( X_val ,pad)
        x_shp,w_shp  = Convolution2D.__get_shape__(X_val,w_val)
        res_shp    = Convolution2D. __assert_shape__(x_shp,w_shp,stride)
        if res_shp:
           
            indices = np.arange(w_shp[1])*(w_shp[2]*w_shp[3])
            
            t0 = time.time()
            X_res = im2col_cython.im2col_2d(np.array(w_shp),np.array(x_shp),stride,X_val.reshape(x_shp[0]*x_shp[1]*x_shp[2],x_shp[3]))
            t1 = time.time()
            print('Cython way im2col',t1-t0)
            # 19600x2500
            
            t0 = time.time()
            w_val_res = w_val.reshape(w_shp[0],w_shp[1]*w_shp[2]*w_shp[3])
            t1 = time.time()
            print('W_val reshape',t1-t0)
            #25x2500
            
            t0 = time.time()
            grad = grad.reshape(x_shp[0],w_shp[0],res_shp[2]*res_shp[3]).swapaxes(1,2).reshape(x_shp[0]*res_shp[2]*res_shp[3],w_shp[0])
            t1 = time.time()
            print('grad reshaping',t1-t0)
            # 19600x25
            
            t0 = time.time()
            w_grad = np.dot(grad.T,X_res)
            t1 = time.time()
            print('Dot for finding w_grad',t1-t0)
            
            
            t0 = time.time()
            x_grad = np.dot(grad, w_val_res)
            t1 = time.time()
            print('Dot prod for xgrad',t1-t0)
            #19600x2500
            
            X_grad = np.zeros(x_shp)
            
            
            t0 = time.time()
            X_grad = col2im_cython.col2im(np.array(w_shp),np.array(x_shp),stride,x_grad)
            t1 = time.time()
            print('Cython way col2im',t1-t0)
            #100*100*31x31
            
            
            t0 = time.time()
            X_grad= X_grad.reshape(x_shp)
            w_grad = w_grad.reshape(w_shp)
            t1 = time.time()
            print('Final Reshaping',t1-t0)
            
      

            return(X_grad,w_grad)
      
            
            

# Testing speed difference between Naive and fast convolutions

The fast convolutions were implemented by im2col_cython - my personal code.

In [25]:
# Testing and benchmarking forwarprop and backprop of fast vs naive
import numpy as np
import time 
np.random.rand(1)
xval = np.random.rand(100, 10, 31, 31)
wval = np.random.rand(25, 10, 5,5)
x1 = Tensor(xval)
w1 = Tensor(wval)
t0 = time.time()
print('In Optimized Forward_prop ' )
Res1 = Convolution2D.__conv_v5_3d__(x1,w1,stride=2,pad=0)
t1 = time.time()
print('Optimized Forward_prop total time: ',t1-t0 )
#"""
t0 = time.time()
Res2 = Convolution2D.__conv_v2_3d__(x1,w1,stride=2,pad=0)
t1 = time.time()
print('\nNaive Forward_prop',t1-t0 )
#print(Res1 - Res2)
assert(abs(Res1-Res2)<10**-8).all()
#"""
#"""
t0 = time.time()
bk1 = Convolution2D.__back_conv_v2_3d__(x1,w1,stride=2,pad=0,grad=Res1)

t1 = time.time()
print('\nNaive Back_prop total time',t1-t0,'\n' )
#"""
print('In Optimized Back_prop') 
t0 = time.time()
bk2 = Convolution2D.__back_conv_v5_3d__(x1,w1,stride=2,pad=0,grad=Res1)
t1 = time.time()
print('Optimized Back_prop total time ',t1-t0) 
#"""
assert(abs(bk2[0]-bk1[0])<10**-9).all()

In Optimized Forward_prop 
Preliminary : getvalue  0.0
Preliminary : pad  0.008035421371459961
Preliminary : getshape  0.0
Preliminary : assert shape  0.0
Cython way im2col: 0.029267072677612305
Dot product: 0.031977176666259766
Final Swapping,reshaping: 0.0
Optimized Forward_prop total time:  0.07825160026550293

Naive Forward_prop 1.3777029514312744

Naive Back_prop total time 4.454310417175293 

In Optimized Back_prop
Cython way im2col 0.028493165969848633
W_val reshape 0.0
grad reshaping 0.0
Dot for finding w_grad 0.019978761672973633
Dot prod for xgrad 0.04398965835571289
Cython way col2im 0.03198885917663574
Final Reshaping 0.0
Optimized Back_prop total time  0.14444375038146973


# Testing speed difference between Naive and fast convolutions

The fast convolutions were implemented by im2col6 - CS231n code.


In [29]:
# Testing and benchmarking forwarprop and backprop of fast vs naive -CS231n code
import numpy as np
import time 
np.random.rand(1)
xval = np.random.rand(100, 10, 31, 31)
wval = np.random.rand(25, 10, 5,5)
x1 = Tensor(xval)
w1 = Tensor(wval)
t0 = time.time()
print('In Optimized Forward_prop ' )
Res1 = Convolution2D.__conv_v5_3d__(x1,w1,stride=2,pad=0)
t1 = time.time()
print('Optimized Forward_prop total time: ',t1-t0 )
#"""
t0 = time.time()
Res2 = Convolution2D.__conv_v2_3d__(x1,w1,stride=2,pad=0)
t1 = time.time()
print('\nNaive Forward_prop',t1-t0 )
#print(Res1 - Res2)
assert(abs(Res1-Res2)<10**-8).all()
#"""
#"""
t0 = time.time()
bk1 = Convolution2D.__back_conv_v2_3d__(x1,w1,stride=2,pad=0,grad=Res1)

t1 = time.time()
print('\nNaive Back_prop total time',t1-t0,'\n' )
#"""
print('In Optimized Back_prop') 
t0 = time.time()
bk2 = Convolution2D.__back_conv_v5_3d__(x1,w1,stride=2,pad=0,grad=Res1)
t1 = time.time()
print('Optimized Back_prop total time ',t1-t0) 
#"""
assert(abs(bk2[0]-bk1[0])<10**-9).all()

In Optimized Forward_prop 
Preliminary : getvalue  0.0
Preliminary : pad  0.0025606155395507812
Preliminary : getshape  0.0
Preliminary : assert shape  0.0
Cython way im2col: 0.055677175521850586
Dot product: 0.03352022171020508
Final Swapping,reshaping: 0.0
Optimized Forward_prop total time:  0.09977841377258301

Naive Forward_prop 1.380481481552124

Naive Back_prop total time 4.1042468547821045 

In Optimized Back_prop
Cython way im2col 0.032623291015625
W_val reshape 0.0
grad reshaping 0.003998756408691406
Dot for finding w_grad 0.016507387161254883
Dot prod for xgrad 0.052613258361816406
Cython way col2im 0.02958989143371582
Final Reshaping 0.0
Optimized Back_prop total time  0.15131187438964844


# Results
Naive implementations are <b>15 to 20</b> times slower than fast imlementations.<br>
Also fast implementaion using im2col_cython is faster than CS231n course code by <b>1.26 times</b>.